In [1]:
!pip install ijson


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 9.3 MB/s eta 0:00:00


In [2]:
!pip install reportlab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.9 MB/s eta 0:00:00


In [5]:
!pip install bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.9 MB/s eta 0:00:00


In [7]:
!pip uninstall -y bitsandbytes


Found existing installation: bitsandbytes 0.49.1
Uninstalling bitsandbytes-0.49.1:
  Successfully uninstalled bitsandbytes-0.49.1


In [8]:
!pip install -U bitsandbytes


  Using cached bitsandbytes-0.49.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.49.1-py3-none-manylinux_2_24_x86_64.whl (59.1 MB)


In [1]:
import json, re, textwrap
from typing import Iterator, Dict, Any, List, Tuple

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import ijson

from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter

# -----------------------------
# Config
# -----------------------------
IN_PATH  = "/content/shap_tokenwise_new.jsonl"
PDF_PATH = "/content/llm_report.pdf"

MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2"

BATCH_SIZE     = 4
MAX_NEW_TOKENS = 400

TOP_K_TOKENS   = 7
LOG_MAX_CHARS  = 900
MAX_INPUT_LEN  = 2048

_ws_re = re.compile(r"\s+")

# -----------------------------
# Stream reader (JSONL or JSON array)
# -----------------------------
def stream_records(path: str) -> Iterator[Dict[str, Any]]:
    with open(path, "r", encoding="utf-8") as f:
        first = f.read(1)
        f.seek(0)
        if first == "[":
            for obj in ijson.items(f, "item"):
                if isinstance(obj, dict):
                    yield obj
        else:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                if line.endswith(","):
                    line = line[:-1]
                try:
                    obj = json.loads(line)
                except Exception:
                    continue
                if isinstance(obj, dict):
                    yield obj

def get_seq_id(rec: Dict[str, Any]) -> str:
    return str(rec.get("sequence_id", rec.get("seq_id", rec.get("id", "Unknown"))))

def clip_text(x: Any, max_chars: int) -> str:
    if x is None:
        return ""
    if not isinstance(x, str):
        try:
            x = json.dumps(x, ensure_ascii=False)
        except Exception:
            x = str(x)
    x = x.replace("\r", " ").strip()
    return x if len(x) <= max_chars else x[:max_chars] + "..."

def normalize_log(s: str) -> str:
    s = s.replace("\n", " ").replace("\t", " ")
    return _ws_re.sub(" ", s).strip()

# -----------------------------
# Top tokens (word-level) from top_word_tokens
# -----------------------------
def extract_top_tokens(rec: Dict[str, Any], k: int) -> List[str]:
    tw = rec.get("top_word_tokens", [])
    pairs: List[Tuple[str, float]] = []
    if isinstance(tw, list):
        for it in tw:
            if isinstance(it, dict) and "token" in it:
                t = str(it.get("token", "")).strip()
                if not t:
                    continue
                try:
                    v = float(it.get("sum_abs_shap", 0.0))
                except Exception:
                    v = 0.0
                pairs.append((t, v))
    pairs.sort(key=lambda x: x[1], reverse=True)

    out, seen, seen_upper = [], set(), set()
    for t, _ in pairs:
        tu = t.upper()
        if tu == "UNK" and "UNK" in seen_upper:
            continue
        if t not in seen:
            seen.add(t)
            out.append(t)
            if tu == "UNK":
                seen_upper.add("UNK")
        if len(out) >= k:
            break
    while len(out) < k:
        if "UNK" not in seen_upper:
            out.append("UNK")
            seen_upper.add("UNK")
        else:
            out.append(f"UNK{len(out)}")  # fallback placeholder
    return out[:k]

# -----------------------------
# Build EXACT log references for each token
# -----------------------------
def token_log_examples(log_text: str, token: str, window: int = 38, max_examples: int = 2) -> List[str]:
    needle = "<UNK>" if token.upper() == "UNK" else token
    out = []
    start = 0
    for _ in range(max_examples):
        i = log_text.find(needle, start)
        if i == -1:
            break
        s = max(0, i - window)
        e = min(len(log_text), i + len(needle) + window)
        snippet = log_text[s:e].strip().replace("`", "'")
        if s > 0:
            snippet = "... " + snippet
        if e < len(log_text):
            snippet = snippet + " ..."
        out.append(f"`{snippet}`")
        start = i + len(needle)
        # if UNK, only take first occurrence
        if token.upper() == "UNK":
            break
    return out

def build_issue_block(log_preview: str, top_tokens: List[str]) -> str:
    lines = []
    lines.append("Issue (top anomaly-driving tokens with log reference):")
    for t in top_tokens:
        ex = token_log_examples(log_preview, t)
        if not ex:
            lines.append(f"- {t} → appears in log as `N/A` (token not found in preview).")
        elif t.upper() == "UNK":
            lines.append(f"- {t} → appears in log as {ex[0]}.")
        elif len(ex) == 1:
            lines.append(f"- {t} → appears in log as {ex[0]}.")
        else:
            lines.append(f"- {t} → appears in log as {ex[0]} and {ex[1]}.")
    return "\n".join(lines)

# -----------------------------
# LLM prompt ONLY for Reason + Fix
# -----------------------------
def build_llm_prompt(seq_id: str, log_preview: str, issue_block: str) -> str:
    return f"""
You are an SRE/security assistant explaining an anomalous CI/DevSecOps log.

TASK:
Write ONLY the following two sections:

Reason:
<2–3 short professional sentences. Must reference each and every tokens mentioned in the Issue bullets and connect them to the log behavior.For one token one point. No percentages.>
Suggested fix:
1. <action within 15words>
2. <action within 15words>
3. <action within 15 words>
<END>

STRICT RULES:
- Do NOT write "Sequence ID".
- Do NOT rewrite or repeat the Issue bullets.
-Must provide a Suggested fix; do not write ‘N/A’ under any circumstances, always give valid Suggested fix sentences.
- Do NOT mention any other sequences.
- Keep Reason conciseNumber-wise, for each token, mention the token name and provide a very short, simple and constructive reason..
- End with <END>.

Context:
Sequence ID: {seq_id}
Log preview: {log_preview}

Issue bullets (for reference only; do NOT repeat):
{issue_block}
""".strip()

def trim_to_end_marker(txt: str) -> str:
    pos = txt.find("<END>")
    if pos != -1:
        txt = txt[:pos].strip()
    txt = re.sub(r"(?im)^\s*Sequence ID:.*\n?", "", txt).strip()
    # ensure Suggested fix exists
    if "Suggested fix:" not in txt:
        txt += "\nSuggested fix:\n1. N/A\n2. N/A\n3. N/A"
    return txt

# -----------------------------
# Device / Load model
# -----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

tok = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
if tok.pad_token_id is None:
    tok.pad_token = tok.eos_token

load_kwargs = {}
if device == "cuda":
    load_kwargs.update(dict(
        device_map="auto",
        torch_dtype=torch.float16,
        load_in_4bit=True,
    ))
else:
    load_kwargs.update(dict(
        device_map=None,
        torch_dtype=torch.float32,
    ))

model = AutoModelForCausalLM.from_pretrained(MODEL_ID, **load_kwargs)
model.eval()

gen_cfg = GenerationConfig(
    max_new_tokens=MAX_NEW_TOKENS,
    do_sample=False,
    temperature=0.0,
    top_p=1.0,
    pad_token_id=tok.pad_token_id,
    eos_token_id=tok.eos_token_id,
)

def llm_generate(prompt: str) -> str:
    enc = tok(prompt, return_tensors="pt", truncation=True, max_length=MAX_INPUT_LEN)
    enc = {k: v.to(device) for k, v in enc.items()} # Fixed line
    with torch.no_grad():
        out = model.generate(**enc, generation_config=gen_cfg)
    gen_ids = out[0][enc["input_ids"].shape[1]:]
    txt = tok.decode(gen_ids, skip_special_tokens=True).strip()
    return trim_to_end_marker(txt)

# -----------------------------
# PDF writer
# -----------------------------
c = canvas.Canvas(PDF_PATH, pagesize=letter)
W, H = letter
margin = 40
y = H - margin
line_h = 12

def pdf_write_block(block: str):
    global y
    c.setFont("Helvetica", 10)
    for raw_line in block.splitlines():
        raw_line = raw_line.rstrip()
        if not raw_line:
            y -= line_h
            continue
        for line in textwrap.wrap(raw_line, width=110):
            if y < margin:
                c.showPage()
                y = H - margin
                c.setFont("Helvetica", 10)
            c.drawString(margin, y, line)
            y -= line_h
    y -= line_h

# -----------------------------
# Main loop
# -----------------------------
count = 0
print("\n--- START ---\n")

for rec in stream_records(IN_PATH):
    seq_id = get_seq_id(rec)
    log_preview = normalize_log(clip_text(rec.get("reconstructed_preview", ""), LOG_MAX_CHARS))

    top_tokens = extract_top_tokens(rec, TOP_K_TOKENS)
    issue_block = build_issue_block(log_preview, top_tokens)

    prompt = build_llm_prompt(seq_id, log_preview, issue_block)
    reason_fix = llm_generate(prompt)

    final_block = (
        f"Sequence ID: {seq_id} | Anomalous\n\n"
        f"{issue_block}\n\n"
        f"{reason_fix}"
    ).strip()

    print(final_block)
    print("\n" + "-"*80 + "\n")

    pdf_write_block(final_block)
    count += 1

c.save()
print(f"Done. Printed {count} sequences. Saved PDF: {PDF_PATH}")

Device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



--- START ---

Sequence ID: 10 | Anomalous

Issue (top anomaly-driving tokens with log reference):
- fail → appears in log as `... pass 0 0<UNK>test 52.72 7606.39 463.9 fail 1 1<UNK>package 31.18 4573.59 444.0 f ...` and `... l 1 1<UNK>package 31.18 4573.59 444.0 fail 0 2<UNK>scan 73.88 6601.79 53.`.
- test → appears in log as `test 24.35 4386.77 494.59 pass 0 0<UNK>tes ...` and `... est 24.35 4386.77 494.59 pass 0 0<UNK>test 52.72 7606.39 463.9 fail 1 1<UNK>pack ...`.
- package → appears in log as `... test 52.72 7606.39 463.9 fail 1 1<UNK>package 31.18 4573.59 444.0 fail 0 2<UNK>scan ...`.
- UNK → appears in log as `test 24.35 4386.77 494.59 pass 0 0<UNK>test 52.72 7606.39 463.9 fail 1 1<UNK> ...`.
- 24 → appears in log as `test 24.35 4386.77 494.59 pass 0 0<UNK>test 5 ...`.
- pass → appears in log as `test 24.35 4386.77 494.59 pass 0 0<UNK>test 52.72 7606.39 463.9 fail ...`.
- 444 → appears in log as `... .9 fail 1 1<UNK>package 31.18 4573.59 444.0 fail 0 2<UNK>scan 73.88 6601.79 5